# $\hspace{1.5 cm}$ **Atención: Esta Notebook debe ejecutarse usando Google Colab**

# Práctica 4 - Visión Artificial

**Autores:** José María García Ortiz, Levi Malest Villareal y Ana Gil Molina

## Importación de las librerías necesarias

In [ ]:
import os
import tensorflow as tf
# Cargar modelos comprimidos desde tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

In [ ]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [ ]:
import cv2
import tensorflow_hub as hub
import threading
import ipywidgets as widgets
from IPython.display import display as ipy_display
from io import BytesIO
from functools import partial
from keras.utils import get_custom_objects

## Setup

In [ ]:
# Python ≥3.5 requerido
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 requerido
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version sólo existe en Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 requerido
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU detectada. CNNs serán muy lentas sin GPU.")
    if IS_COLAB:
        print("Cambiar runtime y seleccionar una GPU como acelerador hardware.")

# Import comunes:
import numpy as np
import os
import cv2

# Para hacer el notebook repetible:
np.random.seed(42)
tf.random.set_seed(42)

# Para graficar las figuras:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Dónde salvar las figuras:
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Utilidades para graficar imágenes RGB y/o gris:

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

In [ ]:
from google.colab import drive
import os
import shutil
import gdown
import zipfile

# IDs de los archivos compartidos (reemplaza con los IDs de tus archivos .zip)
file_ids = ['1nK9azVnkKjTMB49v3UcafTQRW20r20Fa', '1HtZK1V6KE7l1QLXs22SYzV0ElUyZHVgM', '1n4o8WcjgqUS8DKqYGrRMciUvYZUWPTdn', '1dTHW4lgvOVX4nuJcOvGPVqLffEREHCKx']

# Nombres locales para los archivos descargados y descomprimidos
download_paths = ['/model.zip', '/Estilos OP4G.zip', '/Contenidos OP4G.zip', '/models.zip']
extract_paths = ['', '', '', '']

# Descargar y descomprimir cada archivo
for file_id, download_path, extract_path in zip(file_ids, download_paths, extract_paths):
    # Descargar archivo
    gdown.download(f'https://drive.google.com/uc?id={file_id}', download_path, quiet=False)

    # Descomprimir el archivo ZIP
    with zipfile.ZipFile(download_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
        print(f"Archivos descomprimidos en '{extract_path}'")

## <div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio OP4D (0.2 pts)</strong></div>

Intentar replicar el entrenamiento y posterior prueba del modelo de clasificación para el *dataset Fashion-MNIST* con la arquitectura ResNet-34 del notebook `VC_using_CNNs.ipynb` (en lugar del modelo original).

Para comenzar, cargamos el dataset y separamos los datos en un conjunto de entrenamiento, uno de validación y uno de test.

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

# Normalización estándar y particionamiento de los datos en subconjuntos de train / validación / test:
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

A continuación, definimos las etiquetas que corresponden a las clases del dataset *Fashion-MNIST*. Este conjunto de datos contiene imágenes de ropa etiquetadas con las siguientes categorías:

In [ ]:
# Define the text labels
fashion_mnist_labels = ["T-shirt/top",  # index 0
                        "Trouser",      # index 1
                        "Pullover",     # index 2
                        "Dress",        # index 3
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6
                        "Sneaker",      # index 7
                        "Bag",          # index 8
                        "Ankle boot"]   # index 9

Inspeccionamos algunos ejemplos extraidos del dataset:

In [ ]:
# Plot a random sample of 10 test images, and their ground truth labels:
figure = plt.figure(figsize=(15, 6))
for i, index in enumerate(np.random.choice(X_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    # Display each image
    ax.imshow(np.squeeze(X_test[index]))
    true_index = y_test[index]
    # Set the title for each image
    ax.set_title("{}".format(fashion_mnist_labels[true_index]), color=("green"))
print("X_train.shape:",X_train.shape)

Empezamos a definir el modelo. En primer lugar, creamos una capa de tipo residual, las cuales, como hemos visto en clase, permiten flujos de gradientes más eficientes durante *training*, y posibilitan la construcción de redes más profundas sin *overfitting* ni *gradient overflow/underflow*.

In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

A continuación, se cosntruye el modelo, que consiste en un modelo secuencial definido mediante el método `.add` junto con un bucle `for` para ir añadiendo capas, cada una con el número de filtros deseados.

In [ ]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[224, 224, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))

Con `model.summary()` podemos inspeccionar fácilmente las capas y el número de parámetros de nuestro modelo:

In [ ]:
model.summary()

Como el modelo espera imágenes de entrada con un tamaño de `(224,224,3)`, pero el dataset con el que estamos trabajando en este ejercicio, *Fashion-MNIST*, contiene imágenes en escala de grises con tamaño `(28,28,1)`, debemos redimensionar dichas imágenes a tamaño `(224,224,3)` antes de pasarlas al modelo, de forma que las dimensiones sean compatibles con la arquitectura de la red. Usamos `tf.data.Dataset` para procesar las imágenes de forma más eficiente.

In [ ]:
def preprocess(image, label):
    image_rgb = tf.image.grayscale_to_rgb(image)  # Convertir a 3 canales
    resized_image = tf.image.resize(image_rgb, [224, 224])  # Redimensionar
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

# Crear el dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.map(preprocess).batch(32).prefetch(1)

valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
valid_dataset = valid_dataset.map(preprocess).batch(32).prefetch(1)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.map(preprocess).batch(32).prefetch(1)

Una vez definido el modelo (su arquitectura), simplemente queda:
1. **Compilarlo**: añadir al grafo de computación nodos para computar la función de pérdida, la(s) métrica(s) de funcionamiento, y la lógica de control para el optimizador. Se hace con el método `.compile()`.
2. **Entrenarlo**: se le pasan los ejemplos etiquetados (tanto de entrenamiento como de validación, pero NO de test). Se hace con el método `.fit()`, que desencadena todo el proceso de ajuste iterativo de pesos mediante gradiente descendente.
3. **Evaluarlo**: sobre el subconjunto de test previamente apartado. Se hace con el método `.evaluate()`.

**¡Atención, la siguiente celda tarda casi una hora en colab (con GPU)!**

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_dataset, epochs=10, validation_data=valid_dataset)
score = model.evaluate(test_dataset)

In [ ]:
# Verificar si la carpeta "models" existe, si no, crearla
if not os.path.exists("models"):
    os.makedirs("models")

# Guardar el modelo
model.save("models/resnet34_OP4D.keras")

In [ ]:
# Registrar la capa personalizada
get_custom_objects().update({'ResidualUnit': ResidualUnit})

In [ ]:
# Cargar el modelo
model = keras.models.load_model("models/resnet34_OP4D.keras")

Comparamos la precisión (_accuracy_) sobre los conjuntos de entrenamiento y de test. Es de esperar que la precisión en el conjunto de test sea algo menor, ya que son datos desconocidos para el modelo. Sin embargo, como se ha mantenido separado el conjunto de test, es ya un estimador bastante decente de lo que cabe esperar del modelo:

In [ ]:
train_loss, train_accuracy = model.evaluate(train_dataset)
train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
test_accuracy

Podemos graficar los valores guardados en la variable `history` devuelta por el método `.fit()`, que muestran la evolución del entrenamiento.

In [ ]:
def plot_history(history, samples=10, init_phase_samples=None):
    epochs = history.params['epochs']

    fig, axs = plt.subplots(1,2,figsize=(12,4))

    axs[0].plot(history.history['accuracy'], 'bo-', label='Training accuracy')
    axs[0].plot(history.history['val_accuracy'], 'go-', label='Validation accuracy')
    axs[0].set_title('Training and validation accuracy')
    axs[0].legend()

    axs[1].plot(history.history['loss'], 'b+-', label='Loss')
    axs[1].plot(history.history['val_loss'], 'g+-', label='Validation loss')
    axs[1].set_title('Training and validation loss')
    axs[1].legend()

plot_history(history)

En las anteriores gráficas podemos apreciar como el accuracy en el conjunto de entrenamiento va mejorando a lo largo de las épocas, aunque en el conjunto de validación se queda algo estancado a partir de la segunda época y aumenta muy levemente. Por otro lado, la pérdida en el conjunto de entrenamiento mejora considerablemente a lo largo de las épocas, ya que va disminuyendo, pero en el conjunto de validación se estanca o incluso empeora al aumentar las épocas. Esto podría indicar que sería conveniente entrenar el modelo con menos épocas, ya que no se ven mejoras significativas a partir de la segunda o tercera época.

Finalmente, podemos predecir unas cuantas imágenes del conjunto de test, y comprobar los resultados:

In [ ]:
# Seleccionar un pequeño subconjunto de 15 imágenes aleatorias del test_dataset
test_subset = test_dataset.take(1)  # Esto toma el primer lote de datos

# Convertimos el dataset de test_subset a una lista para que podamos manipular las imágenes
X_test_resized = []
y_test_resized = []
for image_batch, label_batch in test_subset:
    X_test_resized.append(image_batch)
    y_test_resized.append(label_batch)

# Convertimos las listas a arrays
X_test_resized = np.concatenate(X_test_resized, axis=0)
y_test_resized = np.concatenate(y_test_resized, axis=0)

# Realizar predicciones
y_pred = model.predict(X_test_resized)

# Dibujamos una muestra aleatoria de 15 imágenes de test, junto con la etiqueta predicha y la correcta (ground truth):
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(X_test_resized.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(X_test_resized[index]))
    predict_index = np.argmax(y_pred[index])
    true_index = y_test_resized[index]
    ax.set_title("Pred:{} (True:{})".format(fashion_mnist_labels[predict_index],
                                  fashion_mnist_labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))

De las $15$ imágenes seleccionadas, solo $3$ han sido predichas incorrectamente. Esto podría corresponderse con el accuracy obtenido en el conjunto de test, ligeramente inferior a $0.9$.

## <div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio OP4E (0.2 pts)</strong></div>

Ídem para el ejemplo de *transfer learning* de dicho notebook.

En este ejercicio, se pide usar la técnica de *transfer learning*, que consiste en emplear el _backbone_ de una CNN preentrenada para, añadiendo unas pocas capas adicionales, reentrenarla con un _dataset_ completamente nuevo. Ya hemos cargado el dataset *Fashion-MNIST* y separado en conjunto de entrenamiento, validación y prueba en el ejercicio anterior.

Por otro lado, debemos realizar un preprocesamiento de los datos, ya que el modelo que vamos a usar espera imágenes de entrada con un tamaño de `(224,224,3)`, por lo que debemos redimensionar nuestras imágenes a tamaño `(224,224,3)`, para que sus dimensiones sean compatibles con la arquitectura de la red.

Además, realizamos un _data augmentation_, que consiste en hacer pequeños cambios sobre las imágenes de entrada para multiplicar el número de ejemplos a usar en el entrenamiento. En este ejemplo, hacemos recortes aleatorios de las imágenes de entrada, y las _flipamos_ de izquierda a derecha, como esquema básico para realizar dicho _data augmentation_:

In [ ]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label, randomize=False):
    if tf.shape(image)[-1] == 1:  # Convertir a RGB si tiene un solo canal
        image = tf.image.grayscale_to_rgb(image)

    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)

    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [ ]:
batch_size = 32

# Crear el dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(1000).repeat()
train_dataset = train_dataset.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)

valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
valid_dataset = valid_dataset.map(preprocess).batch(batch_size).prefetch(1)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.map(preprocess).batch(batch_size).prefetch(1)

Usaremos **Xception** como CNN _backbone_, la cual es una red con más de 100 capas, cuyos pesos "congelaremos", y a la que simplemente añadiremos una capa `GlobalAveragePooling2D` seguida de una densa (`Dense`) con activación `softmax`:

In [ ]:
n_classes = len(fashion_mnist_labels)

base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

Enumeramos las capas de la red:

In [ ]:
for index, layer in enumerate(base_model.layers):
    print(index, layer.name)

Al estar `congelados` la mayoría de los pesos, el entrenamiento puede ser (relativamente) mucho más rápido (por ahora sólo involucra a la capa superior):

**¡Atención, la siguiente celda tarda unos 20 minutos en colab (con GPU)!**

In [ ]:
dataset_size = len(X_train)
batch_size = 32

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_dataset,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_dataset,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

Evaluamos el modelo sobre el conjunto de test, mediante el método `.evaluate()`:

In [ ]:
score = model.evaluate(test_dataset)

In [ ]:
# Verificar si la carpeta "models" existe, si no, crearla
if not os.path.exists("models"):
    os.makedirs("models")

# Guardar el modelo
model.save("models/Xception_one_layer_OP4E.keras")

In [ ]:
# Cargar el modelo
model = keras.models.load_model("models/Xception_one_layer_OP4E.keras")

Comparamos la precisión (_accuracy_) sobre los conjuntos de entrenamiento y de test. Ya sabemos que es normal que la precisión en el conjunto de test sea algo menor, ya que son datos desconocidos para el modelo. Sin embargo, como se ha mantenido separado el conjunto de test, es un buen estimador para el modelo:

In [ ]:
# Dataset de evaluación sin `.repeat()`
train_eval_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_eval_dataset = train_eval_dataset.map(partial(preprocess, randomize=False)).batch(batch_size).prefetch(1)

train_loss, train_accuracy = model.evaluate(train_eval_dataset)
train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
test_accuracy

A continuación, graficamos los valores guardados en la variable `history` devuelta por el método `.fit()`, que muestran la evolución del entrenamiento.

In [ ]:
def plot_history(history, samples=10, init_phase_samples=None):
    epochs = history.params['epochs']

    fig, axs = plt.subplots(1,2,figsize=(12,4))

    axs[0].plot(history.history['accuracy'], 'bo-', label='Training accuracy')
    axs[0].plot(history.history['val_accuracy'], 'go-', label='Validation accuracy')
    axs[0].set_title('Training and validation accuracy')
    axs[0].legend()

    axs[1].plot(history.history['loss'], 'b+-', label='Loss')
    axs[1].plot(history.history['val_loss'], 'g+-', label='Validation loss')
    axs[1].set_title('Training and validation loss')
    axs[1].legend()

plot_history(history)

En las gráficas anteriores, podemos apreciar que el accuracy en el conjunto de entrenamiento va aumentando a lo largo de las épocas, mientras que en el de validación es variable y no sigue una distribución clara. Vemos que en la segunda época disminuye, luego aumenta en la tercera época, y a partir de ahí empieza a disminuir otra vez. Esto podría ser una señal de overfitting, ya que la precisión en el conjunto de validación no mejora de forma consistente, mientras que en el conjunto de entrenamiento sí lo hace. Por otro lado, observando la gráfica con la pérdida, vemos que en el conjunto de entrenamiento va disminuyendo a lo largo de las épocas, y por tanto mejorando, mientras que en el de validación fluctúa y finalmente empeora. Esto también podría ser un indicio de sobreajuste, ya que el modelo no logra generalizar bien.

Finalmente, podemos predecir unas cuantas imágenes del conjunto de test, y comprobar los resultados:

In [ ]:
test_subset = test_dataset.take(1)  # Esto toma el primer lote de datos

# Convertimos el dataset de test_subset a una lista para que podamos manipular las imágenes y etiquetas
X_test_resized = []
y_test_resized = []
for image_batch, label_batch in test_subset:
    X_test_resized.append(image_batch)
    y_test_resized.append(label_batch)

# Convertimos las listas a arrays
X_test_resized = np.concatenate(X_test_resized, axis=0)
y_test_resized = np.concatenate(y_test_resized, axis=0)

# Realizar predicciones
y_pred = model.predict(X_test_resized)

# Crear una lista con las etiquetas reales y predichas
for i in range(len(y_test_resized)):
    predict_index = np.argmax(y_pred[i])
    true_index = y_test_resized[i]
    pred_label = fashion_mnist_labels[predict_index]
    true_label = fashion_mnist_labels[true_index]

    # Verificar si la predicción es correcta
    if predict_index == true_index:
        print(f"Image {i + 1}: Predicted: {pred_label} (Correct), True: {true_label}")
    else:
        print(f"Image {i + 1}: Predicted: {pred_label} (Incorrect), True: {true_label}")


Vemos que de las $32$ imágenes seleccionadas del conjunto de test, $8$ se han predicho mal, lo cual tiene sentido, ya que antes hemos obtenido que el accuracy en test era de $0.6944$.

Sin embargo, para obtener mejores resultados, conviene ahora "descongelar" los pesos del _backbone_, y hacer un ajuste de todos los pesos de la red.

**¡Atención, la siguiente celda tarda más de media hora en colab (con GPU)!**

In [ ]:
dataset_size = len(X_train)
batch_size = 32

for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                metrics=["accuracy"])
history = model.fit(train_dataset,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_dataset,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=10)

Evaluamos el modelo sobre el conjunto de test, mediante el método `.evaluate()`:

In [ ]:
score = model.evaluate(test_dataset)

In [ ]:
# Verificar si la carpeta "models" existe, si no, crearla
if not os.path.exists("models"):
    os.makedirs("models")

# Guardar el modelo
model.save("models/Xception_all_layers_OP4E.keras")

In [ ]:
# Cargar el modelo
model = keras.models.load_model("models/Xception_all_layers_OP4E.keras")

De nuevo, comparamos la precisión (_accuracy_) sobre los conjuntos de entrenamiento y de test. Vemos que mejoran los valores respecto al modelo anterior, donde usábamos todos los pesos congelados excepto los de la capa superior

In [ ]:
# Dataset de evaluación sin `.repeat()`
train_eval_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_eval_dataset = train_eval_dataset.map(partial(preprocess, randomize=False)).batch(batch_size).prefetch(1)

In [ ]:
train_loss, train_accuracy = model.evaluate(train_eval_dataset)
train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
test_accuracy

Otra vez, graficamos los valores guardados en la variable `history` devuelta por el método `.fit()`, que muestran la evolución del entrenamiento.

In [ ]:
def plot_history(history, samples=10, init_phase_samples=None):
    epochs = history.params['epochs']

    fig, axs = plt.subplots(1,2,figsize=(12,4))

    axs[0].plot(history.history['accuracy'], 'bo-', label='Training accuracy')
    axs[0].plot(history.history['val_accuracy'], 'go-', label='Validation accuracy')
    axs[0].set_title('Training and validation accuracy')
    axs[0].legend()

    axs[1].plot(history.history['loss'], 'b+-', label='Loss')
    axs[1].plot(history.history['val_loss'], 'g+-', label='Validation loss')
    axs[1].set_title('Training and validation loss')
    axs[1].legend()

In [ ]:
plot_history(history)

En las anteriores gráficas observamos que tanto el accuracy como la pérdida en el conjunto de entrenamiento fluctúan, pero sin lograr una gran mejora con las épocas. Lo mismo ocurre con el conjunto de validación. El rendimiento del modelo parece haberse estabilizado. Sin embargo, si lo comparamos con el modelo obtenido al ajustar únicamente los pesos de la capa superior, observamos que los valores obtenidos en este caso son algo mejores.

Finalmente, podemos predecir unas cuantas imágenes del conjunto de test, y comprobar los resultados:

In [ ]:
test_subset = test_dataset.take(1)  # Esto toma el primer lote de datos

# Convertimos el dataset de test_subset a una lista para que podamos manipular las imágenes y etiquetas
X_test_resized = []
y_test_resized = []
for image_batch, label_batch in test_subset:
    X_test_resized.append(image_batch)
    y_test_resized.append(label_batch)

# Convertimos las listas a arrays
X_test_resized = np.concatenate(X_test_resized, axis=0)
y_test_resized = np.concatenate(y_test_resized, axis=0)

# Realizar predicciones
y_pred = model.predict(X_test_resized)

# Crear una lista con las etiquetas reales y predichas
for i in range(len(y_test_resized)):
    predict_index = np.argmax(y_pred[i])
    true_index = y_test_resized[i]
    pred_label = fashion_mnist_labels[predict_index]
    true_label = fashion_mnist_labels[true_index]

    # Verificar si la predicción es correcta
    if predict_index == true_index:
        print(f"Image {i + 1}: Predicted: {pred_label} (Correct), True: {true_label}")
    else:
        print(f"Image {i + 1}: Predicted: {pred_label} (Incorrect), True: {true_label}")

Observamos que al hacer un ajuste más fino, descongelando los pesos del *backbone*, para ajustar todos los pesos de la red, los resultados y las métricas que se obtienen mejoran ligeramente. Sin embargo, esta mejora tampoco es demasiado grande, ya que se aprecia como en el segundo entrenamiento del modelo (con todos los pesos descongelados) el rendimiento se estabiliza y fluctúa, sin lograr grandes mejoras a lo largo de las épocas.

## <div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio OP4F (0.2 pts)</strong></div>

Ídem para el ejemplo de MNIST.

Ya tenemos el dataset *Fashion-MNIST* cargado y separado en conjunto de entrenamiento, validación y prueba del ejercicio OP4D. Por lo tanto, podemos pasar directamente a definir el modelo, compilarlo con el método `.compile()`, entrenarlo con el método `.fit()` y  evaluarlo con el método `.evaluate()`.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

In [ ]:
# Verificar si la carpeta "models" existe, si no, crearla
if not os.path.exists("models"):
    os.makedirs("models")

# Guardar el modelo
model.save("models/CNN_OP4F.keras")

In [ ]:
# Cargar el modelo
model = keras.models.load_model("models/CNN_OP4F.keras")

Comparamos la precisión (_accuracy_) sobre los conjuntos de entrenamiento y de test. De nuevo, obtenemos que la precisión en el conjunto de test es algo menor que en el de entrenamiento, ya que los datos de test son desconocidos para el modelo. Sin embargo, como se ha mantenido separado el conjunto de test, es un buen estimador para el modelo:

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train)
train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
test_accuracy

A continuación, graficamos los valores guardados en la variable `history` devuelta por el método `.fit()`, que muestran la evolución del entrenamiento.

In [ ]:
def plot_history(history, samples=10, init_phase_samples=None):
    epochs = history.params['epochs']

    fig, axs = plt.subplots(1,2,figsize=(12,4))

    axs[0].plot(history.history['accuracy'], 'bo-', label='Training accuracy')
    axs[0].plot(history.history['val_accuracy'], 'go-', label='Validation accuracy')
    axs[0].set_title('Training and validation accuracy')
    axs[0].legend()

    axs[1].plot(history.history['loss'], 'b+-', label='Loss')
    axs[1].plot(history.history['val_loss'], 'g+-', label='Validation loss')
    axs[1].set_title('Training and validation loss')
    axs[1].legend()

plot_history(history)

Observando las anteriores gráficas, apreciamos que el modelo empieza con una precisión de $0.8036$ en la primera época en el conjunto de entrenamiento, y va mejorando progresivamente hasta alcanzar $0.9561$ en la décima época. Esto indica que el modelo está mejorando y aprendiendo con el tiempo. Además, la pérdida de entrenamiento disminuye continuamente hasta el final del entrenamiento, lo cual indica que el modelo está aprendiendo efectivamente durante las épocas. Por otro lado, la precisión en el conjunto de validación aumenta más levemente a lo largo de las épocas, aunque hay una ligera caída en la última época, lo cual podría indicar que el modelo está empezando a sobreajustarse. En cuanto a la pérdida, vemos que en el conjunto de validación sigue una tendencia similar a la precisión, pues mejora levemente con pequeñas fluctuaciones en el camino. Esta muestra que el modelo generaliza bien, aunque las pequeñas fluctuaciones podrían indicar que el modelo está cerca de su límite en cuanto a su capacidad de generalización.

Finalmente, predecimos unas cuantas imágenes del conjunto de test, y comprobamos los resultados:

In [ ]:
y_pred = model.predict(X_test)

# Dibujamos una muestra aleatoria de 15 imágenes de test, junto con la etiqueta predicha y la correcta (ground truth):
figure = plt.figure(figsize=(20, 8))
for i, index in enumerate(np.random.choice(X_test.shape[0], size=15, replace=False)):
    ax = figure.add_subplot(3, 5, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(X_test[index]))
    predict_index = np.argmax(y_pred[index])
    true_index = y_test[index]
    ax.set_title("Pred:{} (True:{})".format(fashion_mnist_labels[predict_index],
                                  fashion_mnist_labels[true_index]),
                                  color=("green" if predict_index == true_index else "red"))

Mirando las predicciones anteriores, podemos observar que de las $15$ imágenes escogidas, tan solo una se ha predicho incorrectamente, lo cual se corresponde con el elevado accuracy obtenido, superior al $0.9$ tanto en el conjunto de entrenamiento como en el de test.

## <div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>Ejercicio OP4G (0.1 pts)</strong></div>

Generar algunas imágenes de ejemplo de arte digital a partir de imágenes propias usando el notebook "Neural Style Transfer" referenciado al final del notebook `VC_using_CNNs.ipynb`.

En este ejercicio se explora una técnica de la visión artificial conocida como ***Neural Style Transfer***, la cual permite combinar dos imágenes de manera que el contenido de una de ellas se mantenga, mientras se aplica el estilo artístico de la otra. Este algoritmo, propuesto en <a href="https://arxiv.org/abs/1508.06576" class="external">A Neural Algorithm of Artistic Style</a> (Gatys et al.), utiliza redes neuronales profundas para transferir el estilo de una imagen (como una obra de arte famosa) a una imagen de contenido (como una fotografía personal).

La idea detrás de *Neural Style Transfer* es optimizar una imagen de salida para que combine las características del contenido de una imagen con las características estilísticas de otra. Para lograr esto, el algoritmo utiliza redes neuronales convolucionales para extraer las estadísticas de contenido y estilo de las imágenes, y luego fusionarlas de manera que la imagen resultante mantenga la estructura de la imagen de contenido pero adquiera el estilo artístico de la imagen de referencia.

In [ ]:
# Cargar el modelo de estilo desde TensorFlow Hub
#hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
hub_model = hub.load('model')

## Ejemplos de Imágenes de Arte Digital



### Funciones auxiliares

Para comenzar, definimos una función `load_img` que carga una imagen desde una ruta, la redimensiona y la convierte en un tensor de TensorFlow con forma $(1, altura, ancho, canales)$, donde la dimensión inicial, que en este caso toma el valor $1$, representa el batch size:

In [ ]:
def load_img(path_to_img):
    '''
    Carga una imagen desde una ruta, la redimensiona y la convierte en un tensor de TensorFlow.

    Parámetros:
        path_to_img: Ruta a la imagen.

    Devuelve:
        img: Tensor de la imagen procesada con forma (1, altura, ancho, canales).
    '''

    # Tamaño máximo para la dimensión más larga de la imagen
    max_dim = 512

    # Leer el archivo desde la ruta como un tensor con 3 canales (RGB) y normalizar a [0, 1]
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    # Dimensiones de la imagen
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    # Nuevas dimensiones de la imagen
    new_shape = tf.cast(shape * scale, tf.int32)

    # Redimensionar la imagen y añadir una nueva dimensión para representar el batch size
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

Por otro lado, se define la función `imshow`, que muestra una imagen usando `Matplotlib`, y de forma opcional le añade un título:

In [ ]:
def imshow(image, title=None):
    '''
    Muestra una imagen usando Matplotlib.

    Parámetros:
        image: Imagen de la forma (altura, ancho, canales) o (1, altura, ancho, canales).
        title (str, opcional): Título de la imagen.
    '''

    # Si la imagen tiene más de 3 dimensiones, se elimina la dimensión del batch size
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    # Mostrar la imagen y el título (si es el caso)
    plt.imshow(image)
    if title:
        plt.title(title)

A continuación, se define la función `tensor_to_image`, que convierte un tensor de `TensorFlow` que representa a una imagen en un objeto de imagen manejable por la librería `PIL`:

In [ ]:
def tensor_to_image(tensor):
    '''
    Convierte un tensor de TensorFlow en un objeto de imagen PIL.

    Parámetros:
        tensor: Tensor de TensorFlow que representa una imagen, con valores en [0, 1].

    Devuelve:
        PIL.Image: Objeto de imagen PIL resultado de convertir el tensor.
    '''

    # Escalar los valores del tensor de [0, 1] a [0, 255]
    tensor = tensor*255

    # Convertir el tensor en un array de NumPy (uint8)
    tensor = np.array(tensor, dtype=np.uint8)

    # Si el tensor tiene más de 3 dimensiones, significa que tiene un batch de imágenes
    if np.ndim(tensor)>3:

        # Asegurarse de que el tensor contenga una sola imagen
        assert tensor.shape[0] == 1

        # Extraer la única imagen del tensor
        tensor = tensor[0]

    # Crear un objeto de imagen PIL a partir del array
    return PIL.Image.fromarray(tensor)

### Ejemplo 1: Estilo del "Guernica" de Picasso

En este ejemplo, utilizamos una imagen propia como imagen de contenido y le aplicamos el estilo artístico de la obra "Guernica" de Pablo Picasso, combinando los detalles de nuestra imagen con el estilo cubista del pintor.

In [ ]:
# Rutas de las imágenes de contenido y de estilo
content_path = 'Contenidos OP4G/Neo_nieve.jpeg'
style_path = 'Estilos OP4G/picasso.png'

In [ ]:
# Cargamos las imágenes como tensores de TensorFlow
content_image = load_img(content_path)
style_image = load_img(style_path)

# Mostramos las imágenes de contenido y de estilo
plt.subplot(1, 2, 1)
imshow(content_image, 'Imagen de contenido')

plt.subplot(1, 2, 2)
imshow(style_image, 'Imagen de estilo')

In [ ]:
# Aplicamos el modelo con las imágenes cargadas y mostramos la imagen resultante
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)

### Ejemplo 2: Estilo abstracto

En este otro ejemplo, utilizamos una imagen propia como imagen de contenido y le aplicamos el estilo abstracto propio del pintor Frankz Kline.

In [ ]:
# Rutas de las imágenes de contenido y de estilo
content_path = 'Contenidos OP4G/estatua_berlin.jpeg'
style_path = 'Estilos OP4G/frankz_kline.jpg'

In [ ]:
# Cargamos las imágenes como tensores de TensorFlow
content_image = load_img(content_path)
style_image = load_img(style_path)

# Mostramos las imágenes de contenido y de estilo
plt.subplot(1, 2, 1)
imshow(content_image, 'Imagen de contenido')

plt.subplot(1, 2, 2)
imshow(style_image, 'Imagen de estilo')

In [ ]:
# Aplicamos el modelo con las imágenes cargadas y mostramos la imagen resultante
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)

## Ejemplo de Vídeos de Arte Digital

Ahora podemos extender la técnica de *Neural Style Transfer* para aplicarla a un vídeo. El proceso consiste en seleccionar una imagen de estilo, la cual se aplicará a cada uno de los frames del vídeo, tratándolos como imágenes de contenido. De esta forma, se obtiene un vídeo que conserva el contenido original, pero transformado con un nuevo estilo artístico.

### Funciones auxiliares

En primer lugar, se define una función que aplica la técnica de transferencia de estilo a un vídeo completo, procesando cada frame por separado. La función, llamada `apply_style_to_video`, carga el modelo de estilo desde `TensorFlow Hub`, lee el vídeo frame a frame, aplica el estilo escogido a cada frame utilizando el modelo, y finalmente guarda el vídeo al que se le ha aplicado el estilo.

In [ ]:
def apply_style_to_video(video_path, style_image_path, output_path):
    """
    Aplica el modelo de transferencia de estilo a un video, frame por frame.

    Parámetros:
        video_path (str): Ruta al video original.
        style_image_path (str): Ruta a la imagen de estilo.
        output_path (str): Ruta donde se guardará el video una vez aplicado el estilo.
    """
    # Cargar el modelo de estilo de TensorFlow Hub
    #hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
    hub_model = hub.load('model')

    # Cargar la imagen de estilo y convertirla en tensor
    style_image = load_img(style_image_path)  # Usar la función `load_img` que definimos antes

    # Leer el video original
    videoreader = cv2.VideoCapture(video_path)
    fps = int(videoreader.get(cv2.CAP_PROP_FPS))  # Frames por segundo
    frame_width = int(videoreader.get(cv2.CAP_PROP_FRAME_WIDTH))  # Ancho del frame
    frame_height = int(videoreader.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Alto del frame

    # Inicializar el writer para guardar el video de salida
    videowriter = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    print("Procesando video...")

    while videoreader.isOpened():
        (grabbed, frame) = videoreader.read()  # Leer el siguiente frame
        if not grabbed:
            break  # Salir cuando no hay más frames

        # Convertir el frame de BGR (OpenCV) a RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Normalizar el frame y agregar dimensión para el batch size
        frame_tensor = tf.convert_to_tensor(frame_rgb, dtype=tf.float32) / 255.0
        frame_tensor = tf.image.resize(frame_tensor, (512, 512))  # Ajustar tamaño
        frame_tensor = frame_tensor[tf.newaxis, ...]

        # Aplicar el modelo de transferencia de estilo
        stylized_frame_tensor = hub_model(frame_tensor, style_image)[0]

        # Convertir el tensor estilizado a imagen numpy
        stylized_frame = tensor_to_image(stylized_frame_tensor)  # Usamos la función `tensor_to_image`

        # Redimensionar el frame estilizado al tamaño original del video
        stylized_frame = np.array(stylized_frame)
        stylized_frame = cv2.resize(stylized_frame, (frame_width, frame_height))

        # Convertir de RGB a BGR (para OpenCV) y escribir el frame procesado
        videowriter.write(cv2.cvtColor(stylized_frame, cv2.COLOR_RGB2BGR))

    # Liberar recursos
    videoreader.release()
    videowriter.release()
    print("Video procesado y guardado en:", output_path)

La función `play_video` nos valdrá para mostrar los vídeos en el propio notebook (sacada del Notebook 01 visto en clase):

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def play_video(videofilename):
    mp4 = open(videofilename,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
      <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

### Ejemplo: Estilo modernista

En este ejemplo, partimos de un vídeo que muestra un paisaje natural y aplicamos el estilo artístico característico del modernismo de Georgia O'Keeffe. Este proceso transforma cada frame del video, combinando la esencia del paisaje original con la expresividad y el estilo del arte de O'Keeffe.

In [ ]:
# Ruta del video original, imagen de estilo y salida
video_path = 'Contenidos OP4G/paisaje.mp4'
style_image_path = 'Estilos OP4G/okeeffe.jpeg'
output_path = 'paisaje_estilizado.avi'

In [ ]:
# Mostramos el vídeo original
play_video(video_path)

In [ ]:
# Mostramos la imagen de estilo
style_image = load_img(style_image_path)
imshow(style_image, 'Imagen de estilo')

**¡Atención, la siguiente celda tarda una media hora en colab!**

In [ ]:
# Aplicar el modelo al video
apply_style_to_video(video_path, style_image_path, output_path)

Usamos el siguiente comando para convertir el vídeo resultante a formato `.mp4`, legible con la librería `OpenCV`:

In [ ]:
!ffmpeg -y -i paisaje_estilizado.avi -vcodec h264 -acodec mp2 paisaje_estilizado.mp4

Para reproducir el vídeo, usamos la función `play_video` sacada del Notebook 01:

In [ ]:
# Mostramos el vídeo resultante
play_video("paisaje_estilizado.mp4")

## Interfaz interactiva con Webcam

**¡Atención, la interfaz con la Webcam no funciona en Colab!**

En esta sección, implementaremos una interfaz interactiva que permita aplicar estilos artísticos en tiempo real a través de la Webcam. Se podrá seleccionar un estilo artístico, y los frames capturados por la cámara se irán transformando, de forma que los elementos del entorno se reinterpretarán con un toque artístico.

In [ ]:
# Variables globales
style_image = None
stop_camera = False
#hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
hub_model = hub.load('model')
image_widget = widgets.Image(format='png')  # Widget para mostrar la imagen de la cámara

En primer lugar, creamos una función `apply_style_to_frame` que aplique el estilo artístico seleccionado a un frame de vídeo o a una imagen utilizando el modelo de transferencia de estilo.

In [ ]:
def apply_style_to_frame(frame):
    """Aplica el estilo seleccionado a un frame."""
    global style_image
    if style_image is None:
        return frame  # Si no hay estilo seleccionado, devuelve el frame original

    # Preprocesar el frame
    content_image = tf.image.convert_image_dtype(frame, tf.float32) # Convertir los valores del frame a [0, 1]
    content_image = tf.image.resize(content_image, (512, 512))      # Redimensionar el frame a 512x512 píxeles
    content_image = content_image[tf.newaxis, :]    # Añadir una nueva dimensión para representar el batch size

    # Aplicar el estilo
    stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

    # Convertir a formato OpenCV
    stylized_image = tf.image.convert_image_dtype(stylized_image, tf.uint8)
    stylized_image = stylized_image.numpy()

    return stylized_image[0]

La siguiente función, `show_image_in_notebook`, se usa para mostrar cada frame en el Jupyter Notebook usando un widget de imagen.

In [ ]:
def show_image_in_notebook(frame):
    """Muestra una imagen en el Jupyter Notebook."""
    _, img_encoded = cv2.imencode('.png', frame)    # Codificar la imagen en formato .png
    img_bytes = img_encoded.tobytes()       # Covertir la imagen codificada a una secuencia de bytes
    img_pil = PIL.Image.open(BytesIO(img_bytes))    # Convertir la secuencia de bytes en una imagen

    # Mostrar imagen en el widget sin limpiar la salida
    with BytesIO() as buffer:
        img_pil.save(buffer, format="PNG")
        image_widget.value = buffer.getvalue()  # Asignar imagen al widget

La función que se define a continuación, `capture_video`, se encarga de capturar el vídeo desde la Webcam, para luego aplicar el estilo artístico seleccionado a cada frame antes de mostrarlo en el Jupyter Notebook.

In [ ]:
def capture_video():
    """Inicia la captura de video desde la Webcam y aplica el estilo en tiempo real."""
    global stop_camera
    videoreader = cv2.VideoCapture(0)

    # Capturar frames de la Webcam hasta que `stop_camera = True`
    while not stop_camera:
        (grabbed, frame) = videoreader.read()   # Leer el siguiente frame
        if not grabbed:
            break

        # Convertir el frame a RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Aplicar el estilo al frame
        styled_frame = apply_style_to_frame(frame)

        # Convertir el frame de vuelta a BGR para mostrarlo con OpenCV
        styled_frame = cv2.cvtColor(styled_frame, cv2.COLOR_RGB2BGR)

        # Mostrar el fotograma estilizado en el widget
        show_image_in_notebook(styled_frame)

    videoreader.release()

La función `start_webcam` se encarga de iniciar la ejecución de la cámara.

In [ ]:
def start_webcam(change):
    """Inicia la ejecución de la cámara."""
    global stop_camera
    stop_camera = False
    thread = threading.Thread(target=capture_video)
    thread.start()

Análogamente, la función `stop_webcam` es la encargada de detener la ejecución de la cámara.

In [ ]:
def stop_webcam(change):
    """Detiene la ejecución de la cámara."""
    global stop_camera
    stop_camera = True

Por último, la función `load_style` tiene como propósito cargar la imagen de estilo seleccionada.

In [ ]:
def load_style(change):
    """Cargar la imagen de estilo."""
    global file_upload
    global style_image

    # Acceder al diccionario que está dentro de la tupla
    uploaded_file = file_upload.value[0]

    # Acceder al contenido de la imagen
    content = uploaded_file['content']

    # Guardar el archivo temporalmente
    with open("temp_style_image.png", "wb") as f:
        f.write(content)

    # Cargar la imagen
    style_image = load_img("temp_style_image.png")
    print("Estilo cargado.")

Una vez definidas todas estas funciones, ya podemos crear la interfaz interactiva. En la siguiente celda se configura dicha interfaz. Mediante `file_upload` se puede cargar la imagen de estilo. Además, los botones `start_button` y `stop_button` permiten iniciar y detener la Webcam, permitiendo la captura de vídeo en tiempo real. Finalmente, los resultados estilizados de la cámara se muestran en tiempo real a través de `image_widget`.

In [ ]:
# Crear widgets
file_upload = widgets.FileUpload(accept='image/*', multiple=False)
start_button = widgets.Button(description="Iniciar Cámara", button_style='success')
stop_button = widgets.Button(description="Detener Cámara", button_style='danger')

# Conectar widgets con funciones
file_upload.observe(load_style, names='value')
start_button.on_click(start_webcam)
stop_button.on_click(stop_webcam)

# Mostrar widgets y el widget de imagen
ipy_display(widgets.HTML("<h3>Interfaz de Estilización en Tiempo Real</h3>"))
ipy_display(widgets.HTML("<p>Carga una imagen para el estilo y presiona 'Iniciar Cámara'.</p>"))
ipy_display(file_upload)
ipy_display(start_button)
ipy_display(stop_button)
ipy_display(image_widget)  # Mostrar el widget de imagen